In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("http://bit.ly/kaggletrain")

In [3]:
df.shape

(891, 12)

In [4]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [5]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [8]:
df = df.loc[df.Embarked.notna(), ["Survived", "Pclass", "Sex", "Embarked"]]

In [9]:
df.shape

(889, 4)

In [10]:
df.head()

,Survived,Pclass,Sex,Embarked
0,0,3,male,S
1,1,1,female,C
2,1,3,female,S
3,1,1,female,S
4,0,3,male,S


In [11]:
X = df.loc[:, ["Pclass"]]
y = df.Survived

In [12]:
X.shape

(889, 1)

In [13]:
y.shape

(889,)

In [14]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver="lbfgs")

In [17]:
from sklearn.model_selection import cross_val_score

cross_val_score(model, X, y, cv=5, scoring="accuracy").mean()

0.6783406335301212

In [18]:
y.value_counts(normalize=True)

0    0.617548
1    0.382452
Name: Survived, dtype: float64

In [19]:
df.head()

,Survived,Pclass,Sex,Embarked
0,0,3,male,S
1,1,1,female,C
2,1,3,female,S
3,1,1,female,S
4,0,3,male,S


In [22]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False)

In [25]:
ohe.fit_transform(df[["Embarked"]])

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [26]:
ohe.categories_

[array(['C', 'Q', 'S'], dtype=object)]

In [27]:
X = df.drop("Survived", axis="columns")
X.head()

,Pclass,Sex,Embarked
0,3,male,S
1,1,female,C
2,3,female,S
3,1,female,S
4,3,male,S


In [28]:
from sklearn.compose import make_column_transformer

In [29]:
column_trans = make_column_transformer(
    (ohe, ["Sex", "Embarked"]), remainder="passthrough"
)

In [31]:
column_trans.fit_transform(X, y)

array([[0., 1., 0., 0., 1., 3.],
       [1., 0., 1., 0., 0., 1.],
       [1., 0., 0., 0., 1., 3.],
       ...,
       [1., 0., 0., 0., 1., 3.],
       [0., 1., 1., 0., 0., 1.],
       [0., 1., 0., 1., 0., 3.]])

In [32]:
from sklearn.pipeline import make_pipeline

In [33]:
pipe = make_pipeline(column_trans, model)

In [35]:
cross_val_score(pipe, X, y, cv=5, scoring="accuracy").mean()

0.7727924839713071